In [ ]:
Скачивание Видео

In [ ]:
import urllib.request

#url = 'https://videos.pexels.com/video-files/3052742/3052742-hd_1920_1080_24fps.mp4'

url = 'https://videos.pexels.com/video-files/3059073/3059073-hd_1920_1080_24fps.mp4'
video_path = 'road_video-2.mp4'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

req = urllib.request.Request(url, headers=headers)

with urllib.request.urlopen(req) as response, open(video_path, 'wb') as out_file:
    data = response.read()
    out_file.write(data)

print("Видео загружено успешно.")

In [ ]:
Обработка видео

In [ ]:
import cv2
import numpy as np

# Путь к видео
video_path = 'road_video-2.mp4'
output_path = 'processed_road_video.mp4'

# Загрузка видео
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 20.0, (1920, 1080))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]

    # Создание маски для нижней половины изображения
    mask = np.zeros_like(frame)
    mask[height//2:, :] = 255

    # Преобразование в цветовое пространство HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Диапазоны для белого и желтого цветов
    white_lower = np.array([0, 0, 200], dtype=np.uint8)
    white_upper = np.array([180, 25, 255], dtype=np.uint8)
    yellow_lower = np.array([18, 94, 140], dtype=np.uint8)
    yellow_upper = np.array([48, 255, 255], dtype=np.uint8)

    # Маски для белого и желтого цветов
    white_mask = cv2.inRange(hsv, white_lower, white_upper)
    yellow_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)

    # Объединяем маски
    combined_mask = cv2.bitwise_or(white_mask, yellow_mask)
    
    # Применение маски для нижней половины изображения
    combined_mask = cv2.bitwise_and(combined_mask, mask[:,:,0])

    # Применение маски к изображению
    masked_edges = cv2.bitwise_and(frame, frame, mask=combined_mask)

    # Преобразование в серый цвет и применение Canny edge detection
    gray = cv2.cvtColor(masked_edges, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    # Определение линии разметки с использованием Hough Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, maxLineGap=50)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)
    
    # Определение направления движения (упрощенный пример)
    direction = "Straight"
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if x1 != x2:  # Простейшая проверка угла наклона
                slope = (y2 - y1) / (x2 - x1)
                if slope > 0.5:
                    direction = "Right"
                elif slope < -0.5:
                    direction = "Left"
    
    cv2.putText(frame, direction, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)

    # Сохранение обработанного кадра
    out.write(frame)

cap.release()
out.release()
print("Видео обработано и сохранено успешно.")